In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [5]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210106025209.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",        "nid": 3}}

env = connections[network]

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [6]:
contracts = {}

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# contracts = {'loans': {'zip': 'core_contracts/loans.zip',
#   'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'},
#  'staking': {'zip': 'core_contracts/staking.zip',
#   'SCORE': 'cx127ab2bd9face92d02c78fff9bc1956657ac2661'},
#  'sicx': {'zip': 'token_contracts/sicx.zip',
#   'SCORE': 'cxec4ef61076fb216c12de32965eee0f9a1c36496a'},
#  'icd': {'zip': 'token_contracts/icd.zip',
#   'SCORE': 'cxe510ed4ae3889d3e289b097226758446774ddbcc'},
#  'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
#   'SCORE': 'cx6a4d643725696fc964874bbb2c7fc56e98651d2d'},
#  'bal': {'zip': 'token_contracts/bal.zip',
#   'SCORE': 'cx5393c47e17123eae5fdda35ba458a31f82127477'}}

In [7]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [8]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [34]:
# Balanced test wallet
btest_wallet = KeyWallet.load("keystores/balanced_test.json", "9re$u{}V-W")

### Send ICX to Baalanced test wallet

In [36]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xf23ceedff6c7da63840071cba575f57ffa1ed0690fdfefdadc25d8a71ba52d04'

In [64]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# contracts['dividends'] = {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'}
deploy = ['loans', 'staking', 'dividends', 'sicx', 'icd', 'dummy_oracle', 'bal']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as contracts_20210108042044.pkl


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx22b895f886907ee00380de11781f7032f856c52b'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxac483355d769d2a7c522a1073205449407aed3af'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxd412155a3917cce1c4def4348a83c766b497bd63'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxb6e499ecf95b01ff6a8d0440e03f3886161fc314'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cx95992cd3399f14cc9681ad68b1048f88f4ffb5ef'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxa86e89905fce9ff75b7cae31855b1e19d35b7e3d'}}

## Deploy All SCOREs

In [16]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx22b895f886907ee00380de11781f7032f856c52b'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxac483355d769d2a7c522a1073205449407aed3af'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxd412155a3917cce1c4def4348a83c766b497bd63'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxb6e499ecf95b01ff6a8d0440e03f3886161fc314'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cx95992cd3399f14cc9681ad68b1048f88f4ffb5ef'}}

### Deploy a single SCORE

In [22]:
# Deploy a single SCORE
score = 'dividends'
loc = {'zip': 'core_contracts/dividends.zip',  'SCORE': ''}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')
res

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


{'txHash': '0xe78a58a4391358e040ce90aa8c3ebe654c6b648848d5eb288a8631fa2a235cb7',
 'blockHeight': 39664,
 'blockHash': '0x23c7fce8fbe8923180f11df68a248e417ea7586869a0c535c034f9ec944e0123',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxa86e89905fce9ff75b7cae31855b1e19d35b7e3d',
 'stepUsed': 1273740000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1273740000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

### Importable methods test module

In [ ]:
api = icon_service.get_score_api(contracts['loans']['SCORE'])

In [ ]:
api

In [ ]:
module = ("from iconsdk.icon_service import IconService\n"
        + "from iconsdk.providers.http_provider import HTTPProvider\n"
        + "from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder\n"
        + "from iconsdk.builder.call_builder import CallBuilder\n"
        + "from iconsdk.signed_transaction import SignedTransaction\n"
        + "icon_service = IconService(HTTPProvider('http://13.57.193.25:9000/', 3))\n"
        + "\n"
        + "class Test(object):\n"
        + "\n"
        + "    def __init__(self):\n"
        + "        pass\n"
        + "\n")
filename = 'methods.py'
for item in api:
    if item.get('readonly', '') == '0x1' and item['type'] == 'function':
        func_params = [f"{param['name']}" for param in item['inputs']]
        params = "{" + ", ".join([f"\'{param}\': {param}" for param in func_params]) + "}"
        module += (f"    def {item['name']}({', '.join(['self'] + func_params)}):\n"
                 + f"        call = CallBuilder().from_('{wallet.get_address()}')\\\n"
                 + f"            .to('{contracts['loans']['SCORE']}')\\\n"
                 + f"            .method('{item['name']}')\\\n"
                 + f"            .params({params})\\\n"
                 + f"            .build()\n"
                 + f"        return icon_service.call(call)\n"
                 + f"\n")
with open(filename, 'w') as f:
    f.write(module)

from methods import Test

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
tests = Test()

In [ ]:
tests.name()
tests.asset_count()
int(tests.get_total_collateral(), 0) / ICX
tests.get_available_assets()
tests.get_account_positions(wallet.get_address())

## Update a SCORE

In [65]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


{'txHash': '0xd00e4d91475d8536bf8f715f25209291d0f3f1caaf8fb8715b2e6fbcfed6a006',
 'blockHeight': 89521,
 'blockHash': '0xc542d6dd5e495d093f4d7b603ac4ae9dd8732a4093da37f7cee101ef7731c047',
 'txIndex': 1,
 'to': 'cx22b895f886907ee00380de11781f7032f856c52b',
 'scoreAddress': 'cx22b895f886907ee00380de11781f7032f856c52b',
 'stepUsed': 2374215200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2374215200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Set parameters

In [23]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'icd', 'method': 'set_oracle_address', 'params':{'_address': contracts['dummy_oracle']['SCORE']}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
#             {'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
#             {'contract': 'sicx', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE']}},
            {'contract': 'loans', 'method': 'set_dividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'sicx', 'method': 'set_staking_address', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [24]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}




Calling setAdmin, with parameters {'_admin': 'cxac483355d769d2a7c522a1073205449407aed3af'} on the sicx contract.
Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'txHash': '0x6db4ff880a9dfba140e5d5666ef23ae541a8df2ea5eed1727212132ad23b72c5', 'blockHeight': 39686, 'blockHash': '0xdcc7c42fbe753e77c6071daf677fbd3d740d97702033eb25d78c1aeeb6a38372', 'txIndex': 1, 'to': 'cx22b895f886907ee00380de11781f7032f856c52b', 'stepUsed': 150320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 150320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'txHash': '0xb6081eff8890fd4b9bb8f5ce786f34bfd71e5624cf5c2520d985ed2c7c40e5f6', 'blockHeight': 39687, 'blockHash': '0x6be0bb3c99a405a2e45a50dd2673be2fed21d0f069846ef8ba84a3f9eba5d43b', 'txIndex': 1, 'to': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'stepUsed': 146480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 146480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Run through read-only methods on all SCOREs

### A single read-only method

In [ ]:
params = {'_owner': wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [13]:
ro_methods = {"loans": ["name", "get_available_assets", "asset_count", "get_total_collateral"],
              "staking": ["name", "get_sICX_address", "get_rate"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [67]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
Address: cx22b895f886907ee00380de11781f7032f856c52b
name: BalancedLoans
get_available_assets: ['{"symbol":"sICX","address":"cxcb61d8a122f1c805b1072bcd5e981e48a2a734a6","peg":"sICX","added":1609979563235995,"active":true}', '{"symbol":"ICD","address":"cx686c34a2bea90b54dd332e058372cd3e65a3d893","peg":"USD","added":1609979565236621,"active":true}', '{"symbol":"sICX","address":"cx812d8183d6b8cb09683cf91503b75feddc2a1fa8","peg":"sICX","added":1609979885492221,"active":true,"bad_debt":782769000000000000000}', '{"symbol":"ICD","address":"cxd412155a3917cce1c4def4348a83c766b497bd63","peg":"USD","added":1609979889495867,"active":true,"bad_debt":200000000000000000000}']
asset_count: 0x4
get_total_collateral: 1607.769

SCORE: staking
Address: cxac483355d769d2a7c522a1073205449407aed3af
name: Staking
get_sICX_address: cx812d8183d6b8cb09683cf91503b75feddc2a1fa8
get_rate: 1.0

SCORE: sicx
Address: cx812d8183d6b8cb09683cf91503b75feddc2a1fa8
name: StakedICX
symbol: sICX
decimals: 18
total

## Deposit ICX and Mint ICD Loan

In [28]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x8255c6752a998ac050f06e12ee7e60b884206b53ed9846ec5de677ac2d221dd2', 'blockHeight': 39732, 'blockHash': '0x09af46644fa8835f1847a49918f3ffb6e47463480e2f543ec9e15f5d9836194b', 'txIndex': 1, 'to': 'cxac483355d769d2a7c522a1073205449407aed3af', 'stepUsed': 570740, 'stepPrice': 10000000000, 'cumulativeStepUsed': 570740, 'eventLogs': [{'scoreAddress': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'indexed': ['OraclePrice(str,int,int)', 'ICXUSD', '0x2454d8dcb2cf44bb', '0x5b844ad04483f'], 'data': []}, {'scoreAddress': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x22b1c8c1227a00000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x22b1c8c1227a00000', '0x4e6f6e65']}, {'scoreAddres

### Just deposit collateral

In [30]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0xbb43804179b271b04aef40cb47a427c8d599c8a017011315f87a94058cdef397', 'blockHeight': 39760, 'blockHash': '0x0953ce10c1c84edb9b1e6d69b812c2fba274d1eb81c9899477618fc47fc7819d', 'txIndex': 1, 'to': 'cxac483355d769d2a7c522a1073205449407aed3af', 'stepUsed': 338900, 'stepPrice': 10000000000, 'cumulativeStepUsed': 338900, 'eventLogs': [{'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'cx22b895f886907ee00380de11781f7032f856c52b', '0x1a055690d9db80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Mint(Address,int,bytes)', 'cx22b895f886907ee00380de11781f7032f856c52b'], 'data': ['0x1a055690d9db80000', '0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72

### Repay Some ICD

In [31]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0xb60269ffb5e3dc0a954bfe6a75ab5f381c72d400323bbe349ae09ce5ae6532e6', 'blockHeight': 39762, 'blockHash': '0x3aedc0330eed74cde46e128c6dfab939e1ad23df15f1f23ff5373ef6f5fb5e08', 'txIndex': 1, 'to': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'stepUsed': 328720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 328720, 'eventLogs': [{'scoreAddress': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'indexed': ['Burn(Address,int)', 'cx22b895f886907ee00380de11781f7032f856c52b'], 'data': ['0x4563918244f40000']}, {'scoreAddress': 'cx22b895f886907ee00380de11781f7032f856c52b', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'ICD', '0x4563918244f40000'], 'data': ['Loan of 5.0 ICD repaid to Balanced.']}, {'scoreAddress': 'cxd412155a3917cce1c4def4348a83c766b497bd63', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx22b895f886907ee00380de11781f7032f856c52b', '0x4563918244f4

### Withdraw Some Collateral

In [32]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdraw_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0x30adf52b4e2bca1414f92c67e2106ca6ced405c0e5a3605352455b30bb834f7c', 'blockHeight': 39765, 'blockHash': '0x24f0efe7b2685743daacb402b2a53edd760aec7328d41069986d12be322ffa24', 'txIndex': 1, 'to': 'cx22b895f886907ee00380de11781f7032f856c52b', 'stepUsed': 217000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 217000, 'eventLogs': [{'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx22b895f886907ee00380de11781f7032f856c52b', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x4563918244f40000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx22b895f886907ee00380de11781f7032f856c52b', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x4563918244f40000'], 'data': ['Collateral withdrawn. 5000000000000000000 sICX sent to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb.']}], 'logsBloom': b'\x00\x00\x00\x00\x08\x00\x00\x00 \x00\x00 \x00\x00\x00\x00\x00\x00\x00\x

### Test Liquidation

In [37]:
# Deposit collateral to fresh wallet
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x6c6e2858ffc1bd6494c9593d5a3e076db639da1e7c301ae8205bd40de32470f9', 'blockHeight': 42456, 'blockHash': '0x1c39a960377adb136e10376f6e2539933755e50ea6c9d1267013f157167b77ed', 'txIndex': 1, 'to': 'cxac483355d769d2a7c522a1073205449407aed3af', 'stepUsed': 349920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 349920, 'eventLogs': [{'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'cx22b895f886907ee00380de11781f7032f856c52b', '0x2a6f1a22364bbe8000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Mint(Address,int,bytes)', 'cx22b895f886907ee00380de11781f7032f856c52b'], 'data': ['0x2a6f1a22364bbe8000', '0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f

#### Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [66]:
params = {'_position': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x1306e03430220e958b5f9b288505620148459784db8f3b944ed8f39cd2fb02b8', 'blockHeight': 89524, 'blockHash': '0x35434b7d7bb3cbbf652c2bbfe1854470407769be9a08efb84bbc06e49023ca2b', 'txIndex': 1, 'to': 'cx22b895f886907ee00380de11781f7032f856c52b', 'stepUsed': 237440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 237440, 'eventLogs': [{'scoreAddress': 'cx812d8183d6b8cb09683cf91503b75feddc2a1fa8', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx22b895f886907ee00380de11781f7032f856c52b', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x0'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx22b895f886907ee00380de11781f7032f856c52b', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x0'], 'data': ['Liquidation reward of 0 sICX sent to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb.']}, {'scoreAddress': 'cx22b895f886907ee00380de11781f7032f856c52b', 'indexed': ['Liquidate(Address,int,str)', 'hx3f01840a599da07b0f620eeae7aa